In [ ]:
# STEP 1  get image from google drive
# STEP 2. resize and covert rgb
# STEP 3. Set one-hot encoding
# STEP 4. Save as numpy array

from PIL import Image
import os, glob, numpy as np
from sklearn.model_selection import train_test_split

caltech_dir = "./drive/My Drive/images/New/clear_image/"
categories = ["cocaspaniel" , "Husky", "Maltese", "Pome", "Retriever","WelshiCorgi","bichon","chihuaua","Bulldog","Poodle","Dalmatian","Dachshund","ShibaInu","Beagle","Pug","Yorkshireterrier"]
#categories = ["cocaspaniel" , "Husky", "Maltese", "Pome", "Retriever","WelshiCorgi","bichon"]
nb_classes = len(categories)

image_w = 160
image_h = 160

pixels = image_h * image_w * 3

X = []
y = []

for idx, cat in enumerate(categories):
    
    #one-hot 돌리기.
    label = [0 for i in range(nb_classes)]
    label[idx] = 1

    image_dir = caltech_dir + "/" + cat
    print(image_dir)
    files = glob.glob(image_dir+"/*.jpg")
    print(cat, " 파일 길이 : ", len(files))
    for i, f in enumerate(files):
        #print(i, f)
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)

        X.append(data)
        y.append(label)


X = np.array(X)
y = np.array(y)



X_train, X_test, y_train, y_test = train_test_split(X, y)

np.save("./drive/My Drive/numpy_data/multi_imag_X_train.npy", X_train)
np.save("./drive/My Drive/numpy_data/multi_image_X_test.npy", X_test)
np.save("./drive/My Drive/numpy_data/multi_image_y_train.npy", y_train)
np.save("./drive/My Drive/numpy_data/multi_image_y_test.npy", y_test)

print("ok", len(y))


In [ ]:
#STEP1 GET numpy array from google drive
#STEP2 data /255 
#STEP3 Featcher batch
#STEP4 Get base model MobileNetV2
#STEP5 Get global featcher average
#STEP6 Prediction layer add ( len categories ) 
#STEP7 Train dataset to model

import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3

config =  tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

train_examples = np.load('./drive/My Drive/numpy_data/multi_imag_X_train.npy', allow_pickle=True)
test_examples = np.load('./drive/My Drive/numpy_data/multi_image_X_test.npy', allow_pickle=True)
train_labels = np.load('./drive/My Drive/numpy_data/multi_image_y_train.npy', allow_pickle=True)
test_labels = np.load('./drive/My Drive/numpy_data/multi_image_y_test.npy', allow_pickle=True)

train_examples = train_examples.astype(float) / 255
test_examples = test_examples.astype(float) / 255
#X_train, X_test, y_train, y_test = np.load('./drive/My Drive/numpy_data/multi_image.npy', allow_pickle=True)

train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))

BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

train_batches = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_batches  = test_dataset.batch(BATCH_SIZE)

for image_batch, label_batch in train_batches.take(1):
   pass

image_batch.shape

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


feature_batch = base_model(image_batch)
print(feature_batch.shape)

base_model.trainable = False

#base_model.summary()

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

prediction_layer = tf.keras.layers.Dense(len(categories),activation='softmax')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)


model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()
len(model.trainable_variables)

initial_epochs = 10
validation_steps=20
model_dir = './drive/My Drive/model'
with tf.device('/device:GPU:0'):    
  if not os.path.exists(model_dir):
    os.mkdir(model_dir)
  model_path = model_dir + '/multi_img_classification.model'
  checkpoint = ModelCheckpoint(filepath=model_path , monitor='val_loss', verbose=1, save_best_only=True)

  early_stopping = EarlyStopping(monitor='val_loss', mode='auto', patience=7)
  history = model.fit(train_batches,
                    epochs=initial_epochs,
                    validation_data=test_batches,
                    callbacks=[checkpoint, early_stopping]
                    )
  
  

In [ ]:
# draw train accuracy, validation accuracy
# draw train loss, validation loss 

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

NameError: ignored

In [ ]:
# go to fine tuning
# down learning rate
# learning start ex-train epoc to fine_tune_epochs
# callback method save model if upgrade

base_model.trainable = True

fine_tune_at = 100

#fine_tune_at` 층 이전의 모든 층을 고정
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

# 학습률 낮춤
base_learning_rate = base_learning_rate/10
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_batches,
                         epochs=total_epochs,
                         initial_epoch =  history.epoch[-1],
                         validation_data=test_batches,
                         callbacks=[checkpoint, early_stopping]
                         )

In [ ]:

# get model from google drive

from keras.models import load_model
model = load_model('./drive/My Drive/model/multi_img_classification.model') # 학습된 모델 
#model = load_model('./multi_img_classification.model') # 학습된 모델 

In [ ]:
from urllib import request
import time
from io import BytesIO
from PIL import Image
import os, glob, numpy as np

categories = ["cocaspaniel" , "Husky", "Maltese", "Pome", "Retriever","WelshiCorgi","bichon","chihuaua","Bulldog","Poodle","Dalmatian","Dachshund","ShibaInu","Beagle","Pug","Yorkshireterrier"]
image_w = 160
image_h = 160

url = input()

# request.urlopen()
res = request.urlopen(url).read()

# Image open
img = Image.open(BytesIO(res))
img = img.convert("RGB")
img = img.resize((image_w, image_h))
data = np.asarray(img)
array = []
array.append(data)
array = np.array(array)
array = array.astype(float) / 255

prediction = model.predict(array)

answer = []
length = len(prediction[0])

for i in range(length):
    tmp = prediction[0][i]
    answer.append([tmp*100,categories[i]])

answer = sorted(answer, reverse =True, key = lambda x:x[0])

result = []
count= 0
for i in answer:
  result.append(i)
  count+=1
  if ( count == 5):
    break

print(result)

